# Table of Contents
 <p><div class="lev1 toc-item"><a href="#基础" data-toc-modified-id="基础-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>基础</a></div><div class="lev2 toc-item"><a href="#基础知识" data-toc-modified-id="基础知识-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>基础知识</a></div><div class="lev2 toc-item"><a href="#结构" data-toc-modified-id="结构-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>结构</a></div><div class="lev3 toc-item"><a href="#边" data-toc-modified-id="边-121"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>边</a></div><div class="lev3 toc-item"><a href="#节点" data-toc-modified-id="节点-122"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>节点</a></div><div class="lev3 toc-item"><a href="#其他" data-toc-modified-id="其他-123"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>其他</a></div><div class="lev3 toc-item"><a href="#内核" data-toc-modified-id="内核-124"><span class="toc-item-num">1.2.4&nbsp;&nbsp;</span>内核</a></div><div class="lev2 toc-item"><a href="#常用API" data-toc-modified-id="常用API-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>常用API</a></div><div class="lev3 toc-item"><a href="#图，操作和张量" data-toc-modified-id="图，操作和张量-131"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>图，操作和张量</a></div><div class="lev3 toc-item"><a href="#可视化" data-toc-modified-id="可视化-132"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>可视化</a></div><div class="lev2 toc-item"><a href="#变量作用域" data-toc-modified-id="变量作用域-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>变量作用域</a></div><div class="lev3 toc-item"><a href="#variable_scope" data-toc-modified-id="variable_scope-141"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>variable_scope</a></div><div class="lev3 toc-item"><a href="#name_scope" data-toc-modified-id="name_scope-142"><span class="toc-item-num">1.4.2&nbsp;&nbsp;</span>name_scope</a></div><div class="lev2 toc-item"><a href="#批标准化" data-toc-modified-id="批标准化-15"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>批标准化</a></div>

# 基础

## 基础知识

In [2]:
import tensorflow as tf

t = tf.add(8, 9)
print(t)

Tensor("Add_1:0", shape=(), dtype=int32)


Tensorflow中涉及的运算都在图中，而图的运行只发生在session中。开启session后，就可以用数据填充node，进行运算；关闭session后就不能进行计算了。因此，**会话提供了操作运行和Tensor求值的环境**

In [4]:
a = tf.constant([1.0, 2.0])
b = tf.constant([3.0, 4.0])
c = a * b

#Create a session
sess = tf.Session()

#Compute the tensor c
print(sess.run(c))

sess.close()

[ 3.  8.]


## 结构

### 边

Tensorflow的边有两种连接关系：数据依赖和控制依赖。实线边表示数据依赖，代表数据，即张量。虚线边表示控制依赖，用于控制操作的运行，这类边上没有数据流过，但源节点必须在目的节点开始执行前完成执行

### 节点

节点又称为算子（OP），一般用来表示施加的数学运算，也可以表示数据输入（feed in）的起点以及输出（push out）的终点；或是读取/写入持久变量（persistent variable）的终点。

|类别         |示例             |
|-----------|--------------|
|数学运算  |Add,Sub,Mul,Div,Exp,Log,Greater,Less,Equal...|
|数组运算  |Concat,Slice,Split,Constant,Rank,Shape,Shuffle...|
|矩阵运算  |MatMul,MatrixInverse,MatrixDeterminant...|
|有状态的操作|Variable，Assign，AssignAdd...|
|神经网络构建操作|SoftMax,Sigmoid, ReLU,Convolution2D,MaxPool|
|检查点操作|Save,Restore|
|队列和同步操作|Enqueue,Dequeue,MutexAcquire,MutexRelease...|
|控制张量流动的操作|Merge,Switch,Enter,Leave,NextIteration...|

### 其他

* 图：操作任务可以描述成有向无环图

In [2]:
import tensorflow as tf

matrix1 = tf.constant([[3., 3.]])  #边
matrix2 = tf.constant([[2.],[2.]]) #边

product = tf.matmul(matrix1, matrix2) #节点

* 会话：创建一个会话来启动图，会话提供在图中执行操作的一些方法。一般模式是建立会话（此时是空图），在会话中添加节点和边，形成一张图，然后执行

In [4]:
sess = tf.Session() #空图
result = sess.run([product])
print(result)

[array([[ 12.]], dtype=float32)]


> 会话主要有两个API接口，Extend（在图中添加节点和边），Run（输入计算的节点和填充必要的数据后进行运算，并输出运算结果）

* 设备：Tensorflow可以指定在哪个设备上执行

In [40]:
#with tf.Session() as sess:
#    with tf.device("/gpu:0"): #指定在第一个gpu上执行
#        matrix1 = tf.constant([[3., 3.]]) 
#        matrix2 = tf.constant([[2.],[2.]])
#        product = tf.matmul(matrix1, matrix2) 

* 变量：在图中有固定的位置，不能像张量（边）一样随意流动。创建变量张量，使用```tf.Variable()```构造函数，需要初始值。初始值包括形状（shape）和类型（type）

In [29]:
#标量
state = tf.Variable(0, name="counter")

print(state) #注意这里没有体现出state的实值

<tf.Variable 'counter_16:0' shape=() dtype=int32_ref>


In [9]:
#常量张量
input1 = tf.Variable(3.0)

print(input1)

<tf.Variable 'Variable:0' shape=() dtype=float32_ref>


Tensorflow提供填充机制，可以在构建图时使用```tf.placeholder()```临时替代任意操作的张量，在调用Session对象的```run()```方法去执行图时，使用填充数据作为调用的参数，调用结束后，填充数据就消失

In [2]:
import tensorflow as tf

input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.multiply(input1, input2)

with tf.Session() as sess1:       
    print(sess1.run([output], feed_dict={input1:[7.], input2:[3.]}))
    

[array([ 21.], dtype=float32)]


### 内核

内核（kernel）时能够运行在特定设备（如CPU，GPU）上的一种对操作的实现。同一个操作可能会对应多个内核。

## 常用API

### 图，操作和张量

表示计算的操作对象```tf.Graph```

|操作|描述|
|----|-----|
|tf.Graph.\_\_init\_\_()|创建一个空图|
|tf.Graph.as_default|将某图设为默认图。如果不显示设置，系统会自动设置一个全拒的默认图。在模块范围内定义的节点都默认加入默认图中|
|tf.Graph.device(device_name_or_function)|定义运行图所使用的设备|
|tf.Graph.name_scope(name)|为节点创建层次化的名称|

tf.Operation类表示图中的一个节点，用于计算张量数据。

|操作|描述|
|----|----|
|tf.Operation.name|操作的名称|
|tf.Operation.type|操作的类型|
|tf.Operation.inputs/output|操作的输入与输出|
|tf.Operation.control_inputs|操作的依赖|
|tf.Operation.run(feed_dict=None,session=None)|在会话中运行该操作|
|tf.Operation.get_attr(name)|获得操作的属性|

tf.Tensor类时操作输出的符号句柄

|操作|描述|
|---|--|
|tf.Tensor.dtype|张量的数据类型|
|tf.Tensor.name|名称|
|tf.Tensor.value_index|张量在操作输出中的索引|
|tf.Tensor.graph|张量所在的图|
|tf.Tensor.op|产生该张量的操作|
|tf.Tensor.consumers()|返回使用该张量的操作列表|
|tf.Tensor.eval(feed_dict=None, session=None)|在会话中求张量的值|
|tf.Tensor.get_shape()|返回用于表示张量的形状的类TensorShape|
|tf.Tensor.set_shape(shape)|更新张量的形状|
|tf.Tensor.device|设置计算该张量的设备|

### 可视化

在程序中给必要**节点**添加摘要（summary）。摘要会收集数据，写入事件文件（event file）。tf.summary.FileWriter类用于在目录中创建事件文件，并添加摘要和事件，用来在TensorBoard中展示。

## 变量作用域

### variable_scope

variable_scope主要给variable_name加前缀。

```v=tf.get_variable(name, shape, dtype, initializer) #通过名字创建或者返回一个变量
tf.variable_scope(<scope_name>) #为变量指定命名空间
```

* 获取变量作用域

In [3]:
import tensorflow as tf
with tf.variable_scope("foo") as foo_scope:
    v = tf.get_variable("v", [1])
with tf.variable_scope(foo_scope):
    w = tf.get_variable("w", [1])

* 变量作用域初始化

variable_scope主要用于RNN共享变量的情况

### name_scope

name_scope会影响op_name，不会影响用get_variable创建的变量，但是会印象用过Variable()创建的变量

In [17]:
import tensorflow as tf
with tf.variable_scope("fooo"):
    with tf.name_scope("bar"):
        v = tf.get_variable("v", [1])
        b = tf.Variable(tf.zeros([1]), name = 'b')
        x = 1.0 + v
#assert v.name == "fooo/v:0"
#assert b.name == "fooo/bar/b:0"
#assert x.name == "fooo/bar/add"

with tf.Session() as sess:
    print(v.name)
    print(b.name)
    print(x.name)

fooo/v:0
fooo/bar/b:0
fooo/bar/add:0


* 创建一个新的variable_scope时不需要把reuse属性设置未False，只需要在使用的时候设置为True就可以了。
* TensorFlow里创建变量的两种方式有 tf.get_variable() 和 tf.Variable() ,在 tf.name_scope下时，tf.get_variable()创建的变量名不受 name_scope 的影响，而且在未指定共享变量时，如果重名会报错，tf.Variable()会自动检测有没有变量重名，如果有则会自行处理。

## 批标准化